In [8]:
import pandas as pd
import json

# Specify the path
file_path = "station_information.json"

# Read the data
with open(file_path) as file:
    data = json.load(file)

# data is a dictionary with keys ['data', 'last_updated', 'ttl', 'version']
data = data['data']
# data['data'] is a dictionary with the single key ['stations']
data = data['stations']
# data['stations'] is a list of dictionaries, where each dict is a station
fields = data[0].keys()


print(f"There are {len(data)} stations in NYC.")
print(f"Information available for each station:")
print(fields)

There are 2209 stations in NYC.
Information available for each station:
dict_keys(['external_id', 'region_id', 'lon', 'capacity', 'eightd_station_services', 'has_kiosk', 'name', 'electric_bike_surcharge_waiver', 'rental_methods', 'rental_uris', 'short_name', 'station_type', 'lat', 'station_id', 'eightd_has_key_dispenser'])


In [2]:
from stations.helper import Station

stations = []

# The fields that matter for our simulation
interesting_fields = ['station_id', 'name', 'lat', 'lon', 'capacity']

# Extract the fields that are provided
for station_info in data:
    info_array = [station_info[field] for field in interesting_fields]
    stations.append(Station(*info_array))

def find_by_attr(attr, value):
    """
    Returns the first station that has the given attribute value
    """
    for station in stations:
        if getattr(station, attr) == value:
            return station
    return None

In [4]:
from stations.helper import Location 
import pickle

try:
    with open('station_adjacency', 'rb') as file:
        adj = pickle.load(file)
except:
    adj = { st.number: [] for st in stations }

    for ix1,station1 in enumerate(stations):
        st1 = station1.number
        for station2 in stations[ix1+1:]:
            st2 = station2.number
            if station1.distance(station2) < 350:
                adj[st1].append(st2)
                adj[st2].append(st1)
    
    with open('station_adjacency', 'wb') as file:
        pickle.dump(adj, file)

In [6]:
from stations.helper import Dfs

connected_component = (0,-1)
dfs = Dfs(adj) 

for id in adj:
    size_my_component = dfs.run_dfs(id)
    connected_component = max(connected_component, (size_my_component, id))

print(f"The maximum connected component has size {connected_component[0]}")

The maximum connected component has size 29
